In [1]:
import os
import pandas as pd
import numpy as np

train_data = pd.read_csv("data/training_data_utf8.csv")

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
date = train_data.transaction_ym.apply(pd.to_datetime, format="%Y-%m")
relativeShare = train_data.plot_area * train_data.plot_share

In [3]:
train_data["relativeShare"] = relativeShare
train_data["transaction_month"] = date.dt.month
train_data["transaction_year"] = date.dt.year

In [4]:
train_data_clean = train_data[["price_per_m2", "transaction_year", "relativeShare", "property_type", "cadastral", "contract_type", "land_use", "protection_zone", "building_ban"]]
train_data_clean = train_data_clean.dropna()
train_data_clean.shape

(44260, 9)

In [5]:
data = train_data_clean.drop("price_per_m2", axis=1)
labels = train_data_clean["price_per_m2"].copy()

In [6]:
train_data_cat = train_data_clean[["property_type", "cadastral", "contract_type", "land_use", "protection_zone", "building_ban"]]

In [7]:
train_data_num = train_data_clean[["transaction_year", "relativeShare"]]

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

In [9]:
from future_encoders import ColumnTransformer
from future_encoders import OneHotEncoder

num_attribs = list(train_data_num)
cat_attribs = list(train_data_cat)

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs)
])

In [10]:
data_prepared = full_pipeline.fit_transform(data)
data_prepared

<44260x177 sparse matrix of type '<class 'numpy.float64'>'
	with 354080 stored elements in Compressed Sparse Row format>

In [14]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation", scores.std())

In [12]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=0.5)
ridge_reg.fit(data_prepared, labels)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [13]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(ridge_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
ridge_reg_rmse_scores = np.sqrt(-scores)

In [15]:
display_scores(ridge_reg_rmse_scores)

Scores: [2082.85964015 1526.08674582 2302.96294114 3331.78397284 1891.81241043
 1919.07233389 1109.51294034 2191.2376963   814.8751798   919.14346608]
Mean: 1808.9347326794584
Standard Deviation 717.8021879777373


In [22]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=0.5)
lasso_reg.fit(data_prepared, labels)

Lasso(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [23]:
scores = cross_val_score(lasso_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
lasso_reg_rmse_scores = np.sqrt(-scores)

In [24]:
display_scores(lasso_reg_rmse_scores)

Scores: [2077.80582305 1508.65993324 2285.4602354  3333.07566411 1901.44322406
 1925.5334023  1120.67019106 2189.58101293  824.09527105  931.32995113]
Mean: 1809.7654708326968
Standard Deviation 713.6794454941377


In [25]:
from sklearn.linear_model import ElasticNet

elastic_reg = ElasticNet(alpha=0.5)
elastic_reg.fit(data_prepared, labels)

ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [26]:
scores = cross_val_score(elastic_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
elastic_reg_rmse_scores = np.sqrt(-scores)

In [27]:
display_scores(elastic_reg_rmse_scores)

Scores: [2572.31767826 2134.28529809 2837.04322475 3515.18429232 2094.86866452
 1981.42176015 1203.70508342 2293.40696851  924.65780401  639.93906835]
Mean: 2019.68298423845
Standard Deviation 841.0574119143745


In [30]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2, include_bias=False)
data_poly = poly_features.fit_transform(data_prepared.toarray())

In [31]:
from sklearn.linear_model import 

lin_poly_reg = LinearRegression()
lin_reg.fit(data_poly, labels)

NameError: name 'LinearRegression' is not defined